# FDDHS assignment solution

Christopher Bailey

In [1]:
import string

In [2]:
def mean(values):
    '''Return the mean of a list of numbers'''
    return(sum(values)/len(values))

def median(values):
    '''Return the median of a list of numbers
    NB: Should be corrected for the case of even number of values!
    '''
    return(sorted(values)[len(values) // 2])

## Log file reader for en deltager

In [3]:
def read_log_file(csv_file, field_sep='\t'):
    
    fp = open(csv_file, 'r')
    all_lines = fp.readlines()
    fp.close()
    
    all_RT = []
    all_resp = []
    all_type = []

    for line in all_lines:
        if line[0] != '#':  # if line.startswith('#')
            split_line = line.split(field_sep)
            if split_line[2].startswith('STIM'):
                #
                stim_time = int(split_line[0]) / 10
                if split_line[2][5] in string.ascii_lowercase:
                    stim_type = 'freq'
                else:
                    stim_type = 'rare'
            else:
                #
                resp_time = int(split_line[0]) / 10
                if ((split_line[2][5] == '1' and stim_type == 'freq') or
                    (split_line[2][5] == '2' and stim_type == 'rare')):
                    
                    resp_type = True  # eller 1
                else:
                    resp_type = False  # eller 0                    

                RT = resp_time - stim_time
                all_RT.append(RT)
                all_resp.append(resp_type)
                all_type.append(stim_type)

    return(all_RT, all_resp, all_type)

In [4]:
RTs, resps, types = read_log_file('logs/0012_CGS_2016-09-30.log')

freq_RT = [rt for rt, tp in zip(RTs, types) if tp == 'freq']
rare_RT = [rt for rt, tp in zip(RTs, types) if tp == 'rare']

freq_resp = [rt for rt, tp in zip(resps, types) if tp == 'freq']
rare_resp = [rt for rt, tp in zip(resps, types) if tp == 'rare']

## Find alle logfilerne og læs gruppemedlemskab

In [12]:
from scb_helpers import find, grep

In [22]:
all_logs = find('logs', '*.log')

In [23]:
all_logs[0]

'logs/0026_XKU_2016-10-21.log'

In [21]:
grep('logs/subj_codes.csv', '0025')

'0025_JBA;Control'

## Loop over logfiler

In [45]:
fp = open('results.csv', 'w')

fp.write('SubjectID,Group,Cond,meanRT,medianRT,Accuracy\n')

for log in all_logs:
    RTs, resps, types = read_log_file(log)

    freq_RT = [rt for rt, tp in zip(RTs, types) if tp == 'freq']
    rare_RT = [rt for rt, tp in zip(RTs, types) if tp == 'rare']

    freq_resp = [rt for rt, tp in zip(resps, types) if tp == 'freq']
    rare_resp = [rt for rt, tp in zip(resps, types) if tp == 'rare']
    
    subj_name = log[5:13]
    subj_line = grep('logs/subj_codes.csv', subj_name)
    subj_group = subj_line.split(';')[1]
    
    output = ','.join([subj_name, subj_group, 'Freq',
              str(mean(freq_RT)), str(median(freq_RT)),
                       str(100*mean(freq_resp))]) + '\n'
    fp.write(output)
    
    output = ','.join([subj_name, subj_group, 'Rare',
              str(mean(rare_RT)), str(median(rare_RT)),
                       str(100*mean(rare_resp))]) + '\n'
    fp.write(output)
    
#     fp.write(subj_name, subj_group, 'Freq', mean(freq_RT),
#           median(freq_RT), 100 * mean(freq_resp))
    
fp.close()

In [29]:
log[5:13]

'0012_CGS'

In [44]:
','.join(log[5:13])

'0,0,1,2,_,C,G,S'

In [46]:
str?

Init signature: str(self, /, *args, **kwargs)
Docstring:     
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
Type:           type


## Summary stats with pandas

In [47]:
import pandas as pd

In [48]:
# yes, this is how easy reading a csv-file _really_ can be...
df = pd.read_csv('assignment/results.csv', delimiter=',')

In [49]:
df.head()

,SubjectID,Group,Cond,meanRT,medianRT,Accuracy
0,0026_XKU,Patient,Freq,573.747363,536.4,95.214844
1,0026_XKU,Patient,Rare,688.119922,639.4,85.156250
2,0035_CRF,Control,Freq,495.657422,461.9,96.777344
3,0035_CRF,Control,Rare,577.473828,548.3,88.671875
4,0010_KHI,Patient,Freq,573.142871,530.7,95.410156


In [50]:
# group the numerical values by Group and Condition,
# and display the mean of each
df.groupby(by=['Group', 'Cond']).mean()

meanRT  medianRT   Accuracy
Group   Cond                                 
Control Freq  500.506172   467.180  96.191406
        Rare  579.040293   547.890  88.886719
Patient Freq  579.202925   540.305  94.790039
        Rare  676.630723   638.320  84.707031